In [1]:
params23 ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "dropout": 0.4,
    "n_hidden": 512,
    "batch_size": 512,
    "lr": 0.00426   
    }

In [2]:
# from google.colab import drive
from pathlib import Path
# drive.mount('/content/gdrive', force_remount=True)
%cd "/content/drive/Othercomputers/my_mac/Coding_stuff/Projects/Surface_Predictor"

In [3]:
creds = '...'
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  print('entered')
  cred_path.parent.mkdir(exist_ok=True)
  cred_path.write_text(creds)
  cred_path.chmod(0o600)

In [4]:
# !kaggle competitions download -c career-con-2019 -p ./data/
# !unzip ./data/X_test.csv.zip -d ./data
# !unzip ./data/X_train.csv.zip -d ./data
# !rm ./data/X_train.csv.zip ./data/X_test.csv.zip

In [5]:
%pip install -Uqq pytorch-lightning torchmetrics wandb

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import wandb
from tqdm.auto import tqdm 

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger
import torchmetrics

from pylab import rcParams
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
%matplotlib inline
%config InlinBackend.figure_format = 'retina'
sns.set_style(style= 'whitegrid')
rcParams['figure.figsize'] = 14,9

In [8]:
pl.seed_everything(42)

42

In [9]:
!wandb login

In [10]:
!pwd

In [11]:
x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
x_train

          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
0            0_0          0  ...                 2.1030                -9.7532
1            0_1          0  ...                 1.5064                -9.4128
2            0_2          0  ...                 1.5922                -8.7267
3            0_3          0  ...                 1.0993               -10.0960
4            0_4          0  ...                 1.4689               -10.4410
...          ...        ...  ...                    ...                    ...
487675  3809_123       3809  ...                 2.0115                -9.0063
487676  3809_124       3809  ...                 3.0696                -8.1257
487677  3809_125       3809  ...                 4.2622                -8.1443
487678  3809_126       3809  ...                 4.7130                -9.4435
487679  3809_127       3809  ...                 4.2751               -10.4980

[487680 rows x 13 columns]

In [12]:
x_train.info()

In [13]:
y_train

      series_id  group_id        surface
0             0        13  fine_concrete
1             1        31       concrete
2             2        20       concrete
3             3        31       concrete
4             4        22     soft_tiles
...         ...       ...            ...
3805       3805        55          tiled
3806       3806        67           wood
3807       3807        48  fine_concrete
3808       3808        54          tiled
3809       3809        56       soft_pvc

[3810 rows x 3 columns]

In [14]:
y_train.surface.value_counts().plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Distribution of target (surface) ')
plt.show()

In [15]:
y_train.surface.value_counts().plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Distribution of target (surface) ')
plt.show()

In [16]:
label_encoder =  LabelEncoder()
encoded_labels = label_encoder.fit_transform(y_train.surface)

In [17]:
label_encoder.classes_

array(['carpet', 'concrete', 'fine_concrete', 'hard_tiles',
       'hard_tiles_large_space', 'soft_pvc', 'soft_tiles', 'tiled',
       'wood'], dtype=object)

In [18]:
encoded_labels[:5], list(y_train.surface[:5])

(array([2, 1, 1, 1, 6]),
 ['fine_concrete', 'concrete', 'concrete', 'concrete', 'soft_tiles'])

In [19]:
y_train['label'] = encoded_labels
y_train

      series_id  group_id        surface  label
0             0        13  fine_concrete      2
1             1        31       concrete      1
2             2        20       concrete      1
3             3        31       concrete      1
4             4        22     soft_tiles      6
...         ...       ...            ...    ...
3805       3805        55          tiled      7
3806       3806        67           wood      8
3807       3807        48  fine_concrete      2
3808       3808        54          tiled      7
3809       3809        56       soft_pvc      5

[3810 rows x 4 columns]

In [20]:
x_train

          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
0            0_0          0  ...                 2.1030                -9.7532
1            0_1          0  ...                 1.5064                -9.4128
2            0_2          0  ...                 1.5922                -8.7267
3            0_3          0  ...                 1.0993               -10.0960
4            0_4          0  ...                 1.4689               -10.4410
...          ...        ...  ...                    ...                    ...
487675  3809_123       3809  ...                 2.0115                -9.0063
487676  3809_124       3809  ...                 3.0696                -8.1257
487677  3809_125       3809  ...                 4.2622                -8.1443
487678  3809_126       3809  ...                 4.7130                -9.4435
487679  3809_127       3809  ...                 4.2751               -10.4980

[487680 rows x 13 columns]

In [21]:
#we dont need the first 3 cols
feature_columns = x_train.columns.to_list()[3:]
feature_columns

['orientation_X',
 'orientation_Y',
 'orientation_Z',
 'orientation_W',
 'angular_velocity_X',
 'angular_velocity_Y',
 'angular_velocity_Z',
 'linear_acceleration_X',
 'linear_acceleration_Y',
 'linear_acceleration_Z']

In [22]:
#it looks like the sequences have been grouped for us
x_train.series_id.value_counts()

0       128
2544    128
2532    128
2533    128
2534    128
       ... 
1274    128
1275    128
1276    128
1277    128
3809    128
Name: series_id, Length: 3810, dtype: int64

In [23]:
sequences = []
for series_id, group in x_train.groupby('series_id'):
  sequence_features = group[feature_columns]
  label = y_train[y_train.series_id == series_id].iloc[0].label
  sequences.append((sequence_features, label))

In [24]:
sequences[0]

(     orientation_X  orientation_Y  ...  linear_acceleration_Y  linear_acceleration_Z
 0         -0.75853       -0.63435  ...                2.10300                -9.7532
 1         -0.75853       -0.63434  ...                1.50640                -9.4128
 2         -0.75853       -0.63435  ...                1.59220                -8.7267
 3         -0.75852       -0.63436  ...                1.09930               -10.0960
 4         -0.75852       -0.63435  ...                1.46890               -10.4410
 ..             ...            ...  ...                    ...                    ...
 123       -0.75943       -0.63316  ...                1.79890                -6.8687
 124       -0.75945       -0.63313  ...                2.35620                -6.8926
 125       -0.75949       -0.63309  ...                0.97328                -9.9795
 126       -0.75950       -0.63307  ...                0.34931               -11.9770
 127       -0.75953       -0.63306  ...               

In [25]:
train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
len(train_sequences), len(val_sequences)

(3048, 762)

In [26]:
#sanity check
len(train_sequences), len(val_sequences)
print('all train seq shape check: ',[(train_sequences[i][0].shape) for i in range(len(train_sequences)) if (train_sequences[i][0].shape) != (128, 10)])
# print('all train label shape check: ',[len(train_sequences[i][1]) for i in range(len(train_sequences)) if len(train_sequences[i][1]) != 10])
print('all train seq shape check: ',[(val_sequences[i][0].shape) for i in range(len(val_sequences)) if (val_sequences[i][0].shape) != (128, 10)])
# print('all train label shape check: ',[len(val_sequences[i][1]) for i in range(len(val_sequences)) if len(val_sequences[i][1]) != 10])

In [27]:

class SurfaceDataset(Dataset):

  def __init__(self, sequences) -> None:
      super().__init__()
      self.sequences = sequences

  def __len__(self):
    return len(self.sequences)

  def __getitem__(self, index):
    sequence, label = self.sequences[index]
    return dict(
      sequence=torch.Tensor(sequence.to_numpy()),
      label=torch.tensor(label).long()
    )

In [28]:
class SurfaceDatamodule(pl.LightningDataModule):

  def __init__(self, train_sequences, val_sequences, batch_size):
    super().__init__()
    self.train_sequences = train_sequences
    self.val_sequences = val_sequences
    self.batch_size = batch_size

  def setup(self, stage=None):
    self.train_dataset = SurfaceDataset(self.train_sequences)
    self.val_dataset = SurfaceDataset(self.val_sequences)

  def train_dataloader(self):
      return DataLoader(self.train_dataset, 
                        batch_size=self.batch_size,
                        num_workers=3)
      
  def val_dataloader(self):
      return DataLoader(self.val_dataset,
                        batch_size=self.batch_size,
                        num_workers=2)

In [29]:

class SurfaceModel(nn.Module):

  def __init__(self, n_features, n_classes, n_hidden,dropout, n_layers=3):
    super().__init__()
    print(dropout)
    self.n_fearues = n_features
    self.n_classes = n_classes
    self.n_hidden = n_hidden
    self.n_layers = n_layers
    self.dropout = dropout

    self.lstm = nn.LSTM(
        input_size =self.n_fearues,
        hidden_size = self.n_hidden,
        num_layers = self.n_layers,
        dropout = self.dropout,
        batch_first = True
    )
    self.cassifier = nn.Linear(self.n_hidden, self.n_classes)

  def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden,_) = self.lstm(x)
        out = hidden[-1]
        return self.cassifier(out)

In [30]:
class SurfacePredictor(pl.LightningModule):
  def __init__(self, n_features, n_classes, n_hidden, seq_length, dropout, n_layers=3, lr=0.0001):
    super().__init__()
    self.lr = lr 
    self.seq_length =seq_length
    self.n_features = n_features

    self.model = SurfaceModel(n_features, n_classes, n_hidden, n_layers=n_layers,dropout=dropout)
    self.criterion = nn.CrossEntropyLoss()
    self.train_ac = torchmetrics.Accuracy()
    self.val_ac = torchmetrics.Accuracy()
    self.save_hyperparameters()


  def forward(self, x, labels=None):
    output = self.model(x)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
      return loss, output

    return output


  def training_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('train_loss', loss, prog_bar=True, logger=True)
    self.log('train_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac)

  def validation_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('val_loss', loss, prog_bar=True, logger=True)
    self.log('val_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac, loggits=outputs)


  def validation_epoch_end(self, validation_step_outputs):
    dummyImput = torch.zeros(
        (1, self.seq_length, self.n_features), device=self.device)
    model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
    torch.onnx.export(self, dummyImput, model_filename, opset_version=11)
    # wandb.save(model_filename,)
    outputs = [out['loggits'] for out in validation_step_outputs]
    flattened_outputs = torch.flatten(
          torch.cat(outputs))
    self.logger.experiment.log(
    {'valid/logits': wandb.Histogram(flattened_outputs.to('cpu')),
      'epoch': self.current_epoch}
    )


  def configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr=self.lr)
 

In [31]:
n_epoch = 10
bs = 256
n_features = len(feature_columns)
n_classes = len(label_encoder.classes_)
n_hidden = 128
seq_length= 128

checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4, verbose=True)
progress_bar = TQDMProgressBar(refresh_rate=30)

callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
logger = WandbLogger(name='onnx', save_dir='./logs', project='surface_pred',settings=wandb.Settings(symlink=False))
# wandb.init(settings=wandb.Settings(symlink=False))
trainer = pl.Trainer(gpus = 1,
                     max_epochs = n_epoch,
                     logger= logger,
                     callbacks = callbacks,
                     precision=16)

In [32]:
params23 ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "dropout": 0.4,
    "n_hidden": 512,
    "batch_size": 512,
    "lr": 0.00426   
    }

In [33]:
def train(params, name):

    # set up W&B logger
    wandb_logger = WandbLogger(log_model=True)  # log final model


    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    # setup model - note how we refer to sweep parameters with wandb.config
    model = SurfacePredictor(n_features, n_classes,params['n_hidden'], seq_length, dropout = params['dropout'], lr = params['lr'])

    # early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    logger = WandbLogger(name=name, save_dir='./logs', project='surface_pred')
    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [34]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [35]:
params23 ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 5128,
    "batch_size": 64,
    "lr": 0.01  
    }

In [36]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 5128,
    "batch_size": 64,
    "lr": 0.01  
    }?

In [37]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 5128,
    "batch_size": 64,
    "lr": 0.01  
    }

In [38]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 5128,
    "batch_size": 64,
    "lr": 0.01  
    }??

In [39]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 5128,
    "batch_size": 64,
    "lr": 0.01  
    }

In [40]:
params??

In [41]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.01  
    }

In [42]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [43]:
train(params, 'run_41_w_128nh')

In [44]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [45]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.04  
    }

In [46]:
train(params, 'run_41_w_128nh_04lr')

In [47]:
class SurfacePredictor(pl.LightningModule):
  def __init__(self, n_features, n_classes, n_hidden, seq_length, dropout, n_layers=3, lr=0.0001):
    super().__init__()
    self.lr = lr 
    self.seq_length =seq_length
    self.n_features = n_features

    self.model = SurfaceModel(n_features, n_classes, n_hidden, n_layers=n_layers,dropout=dropout)
    self.criterion = nn.CrossEntropyLoss()
    self.train_ac = torchmetrics.Accuracy()
    self.val_ac = torchmetrics.Accuracy()
    self.save_hyperparameters()


  def forward(self, x, labels=None):
    output = self.model(x)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
      return loss, output

    return output


  def training_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('train_loss', loss, prog_bar=True, logger=True)
    self.log('train_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac)

  def validation_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('val_loss', loss, prog_bar=True, logger=True)
    self.log('val_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac, loggits=outputs)


  def validation_epoch_end(self, validation_step_outputs):
    dummyImput = torch.zeros(
        (1, self.seq_length, self.n_features), device=self.device)
    model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
    # torch.onnx.export(self, dummyImput, model_filename, opset_version=11)
    # wandb.save(model_filename,)
    outputs = [out['loggits'] for out in validation_step_outputs]
    flattened_outputs = torch.flatten(
          torch.cat(outputs))
    self.logger.experiment.log(
    {'valid/logits': wandb.Histogram(flattened_outputs.to('cpu')),
      'epoch': self.current_epoch}
    )


  def configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr=self.lr)
 

In [48]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.04  
    }

In [49]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [50]:
train(params, 'run_41_w_128nh_04lr')

In [51]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":256,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.01  
    }

In [52]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [53]:
train(params, 'run_41')

In [54]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01  
    }
train(params, 'run_41_orig')

In [55]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":256,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01  
    }
train(params, 'run_41_sl256')

In [56]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01  
    }
train(params, 'run_41_orig')

In [57]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":256,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01  
    }
train(params, 'run_41_sl256')

In [58]:
class SurfacePredictor(pl.LightningModule):
  def __init__(self, n_features, n_classes, n_hidden, seq_length, dropout, n_layers=3, lr=0.0001):
    super().__init__()
    self.lr = lr 
    self.seq_length =seq_length
    self.n_features = n_features

    self.model = SurfaceModel(n_features, n_classes, n_hidden, n_layers=n_layers,dropout=dropout)
    self.criterion = nn.CrossEntropyLoss()
    self.train_ac = torchmetrics.Accuracy()
    self.val_ac = torchmetrics.Accuracy()
    self.save_hyperparameters()


  def forward(self, x, labels=None):
    output = self.model(x)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
      return loss, output

    return output


  def training_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('train_loss', loss, prog_bar=True, logger=True)
    self.log('train_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac)

  def validation_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('val_loss', loss, prog_bar=True, logger=True)
    self.log('val_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac, loggits=outputs)


  def validation_epoch_end(self, validation_step_outputs):
    dummyImput = torch.zeros(
        (1, self.seq_length, self.n_features), device=self.device)
    model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
    # torch.onnx.export(self, dummyImput, model_filename, opset_version=11)
    # wandb.save(model_filename,)
    outputs = [out['loggits'] for out in validation_step_outputs]
    flattened_outputs = torch.flatten(
          torch.cat(outputs))
    self.logger.experiment.log(
    {'valid/logits': wandb.Histogram(flattened_outputs.to('cpu')),
      'epoch': self.current_epoch}
    )


  def configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr=self.lr)
 

In [59]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":256,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.01,
    "n_layers" : 2  
    }

In [60]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'],
                             n_layers= params['n_layers'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [61]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":256,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.01,
    "n_layers" : 2  
    }

In [62]:
train(params, 'run_41-2')

In [63]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01  
    }
train(params, 'run_41_orig')

In [64]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'],
                             n_layers= params['n_layers'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()

In [65]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01  
    }
train(params, 'run_41_orig')

In [66]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":256,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01,
    "n_layers" : 2  
    }
train(params, 'run_41_sl256')

In [67]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01,
    "n_layers" : 2   
    }
train(params, 'run_41_orig')

In [68]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.75,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.01,
    "n_layers" : 2   
    }
train(params, 'run_41_do')

In [69]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.5,
    "n_hidden": 128,
    "batch_size": 64,
    "lr": 0.01,
    "n_layers" : 2   
    }
train(params, 'run_41_orig')

In [70]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.75,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.0001,
    "n_layers" : 3   
    }
train(params, 'run_41_yt')

In [71]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'],
                             n_layers= params['n_layers'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()
    return trainer, data_module

In [72]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.75,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.0001,
    "n_layers" : 3   
    }
trainer, data_module = train(params, 'run_41_yt')

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]